Itay Chabra  
Neta Ben Mordechai

# Imports

In [ ]:
#!pip install language_models
!pip install tensorflow-gpu
!pip install --upgrade tensorflow
!pip install keras-preprocessing-gpu
import tensorflow as tf
!pip install tensorboardX

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
ERROR: Could not find a version that satisfies the requirement keras-preprocessing-gpu (from versions: none)
ERROR: No matching distribution found for keras-preprocessing-gpu


In [ ]:
device_name = tf.test.gpu_device_name()
print(device_name)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

/device:GPU:0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
!git clone https://github.com/netabm/language_models.git

Cloning into 'language_models'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 109 (delta 57), reused 105 (delta 55), pack-reused 0 (from 0)
Receiving objects: 100% (109/109), 19.38 MiB | 21.76 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [ ]:
import keras

import keras.backend as K
from keras.datasets import imdb
from keras.layers import  LSTM, Embedding, TimeDistributed, Input, Dense
from keras.models import Model
from tensorflow.python.client import device_lib

from tensorflow.keras.losses import sparse_categorical_crossentropy

from tqdm import tqdm
import os, random

from argparse import ArgumentParser

import numpy as np

from tensorboardX import SummaryWriter

from language_models import util

CHECK = 5

In [ ]:
def generate_seq(model : Model, seed, size, temperature=1.0):
    """
    :param model: The complete RNN language model
    :param seed: The first few wordas of the sequence to start generating from
    :param size: The total size of the sequence to generate
    :param temperature: This controls how much we follow the probabilities provided by the network. For t=1.0 we just
        sample directly according to the probabilities. Lower temperatures make the high-probability words more likely
        (providing more likely, but slightly boring sentences) and higher temperatures make the lower probabilities more
        likely (resulting is weirder sentences). For temperature=0.0, the generation is _greedy_, i.e. the word with the
        highest probability is always chosen.
    :return: A list of integers representing a samples sentence
    """

    ls = seed.shape[0]

    # Due to the way Keras RNNs work, we feed the model a complete sequence each time. At first it's just the seed,
    # zero-padded to the right length. With each iteration we sample and set the next character.

    tokens = np.concatenate([seed, np.zeros(size - ls)])

    for i in range(ls, size):

        probs = model.predict(tokens[None,:])

        # Extract the i-th probability vector and sample an index from it
        next_token = util.sample_logits(probs[0, i-1, :], temperature=temperature)

        tokens[i] = next_token

    return [int(t) for t in tokens]

In [ ]:
def sparse_loss(y_true, y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [ ]:
class Args:
  epochs = 20 # Number of epochs
  embedding_size = 300 # Size of the word embeddings on the input layer.
  out_every = 1 # Output every n epochs.
  lr = 0.001 # Learning rate
  batch = 128 # Batch size
  task = 'wikisimple'
  data = './data' # Data file. Should contain one sentence per line.
  lstm_capacity = 256
  max_length = None # Sentence max length.
  top_words = 10000 # Word list size.
  limit = None # Character cap for the corpus - not relevant in our exercise.
  tb_dir = './runs/words' # Tensorboard directory
  seed = -1 # RNG seed. Negative for random (seed is printed for reproducability).
  extra = None # Number of extra LSTM layers.

options = Args()

In [ ]:
tbw = SummaryWriter(log_dir=options.tb_dir)

if options.seed < 0:
    seed = random.randint(0, 1000000)
    print('random seed: ', seed)
    np.random.seed(seed)
else:
    np.random.seed(options.seed)

if options.task == 'wikisimple':

    x, w2i, i2w = util.load_words(util.DIR + '/datasets/wikisimple.txt', vocab_size=options.top_words, limit=options.limit)

    # Finding the length of the longest sequence
    x_max_len = max([len(sentence) for sentence in x])

    numwords = len(i2w)
    print('max sequence length ', x_max_len)
    print(numwords, 'distinct words')

    x = util.batch_pad(x, options.batch, add_eos=True)

elif options.task == 'file':

    x, w2i, i2w = util.load_words(options.data_dir, vocab_size=options.top_words, limit=options.limit)

    # Finding the length of the longest sequence
    x_max_len = max([len(sentence) for sentence in x])

    numwords = len(i2w)
    print('max sequence length ', x_max_len)
    print(numwords, 'distinct words')

    x = util.batch_pad(x, options.batch, add_eos=True)

else:
    raise Exception('Task {} not recognized.'.format(options.task))

def decode(seq):
    return ' '.join(i2w[id] for id in seq)

print('Finished data loading. ', sum([b.shape[0] for b in x]), ' sentences loaded')

random seed:  959238
raw data read
max sequence length  132
10000 distinct words
max length per batch:  [15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 35, 36, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 42, 42, 43, 44, 45,

# Train Validation Test Split

In [ ]:
def split_dataset(dataset, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    """
    Splits the dataset into training, validation, and test sets.

    Parameters:
    - dataset: List of sequences or sentences
    - train_ratio: The proportion of data for the training set (default 0.8)
    - val_ratio: The proportion of data for the validation set (default 0.1)
    - test_ratio: The proportion of data for the test set (default 0.1)

    Returns:
    - train_data, val_data, test_data: The split datasets
    """
    # Check that the ratios sum to 1
    if train_ratio + val_ratio + test_ratio != 1.0:
        raise ValueError("The sum of train, validation, and test ratios must equal 1.")

    # Get the total number of samples in the dataset
    total_len = len(dataset)

    # Calculate the split lengths
    train_len = int(total_len * train_ratio)
    val_len = int(total_len * val_ratio)

    # Split the dataset without shuffling
    train_data = dataset[:train_len]
    val_data = dataset[train_len:train_len+val_len]
    test_data = dataset[train_len+val_len:]

    return train_data, val_data, test_data

In [ ]:
train_data, val_data, test_data = split_dataset(x, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1)

# Model definition and Training

In [ ]:
## Define model

input = Input(shape=(None, ))
embedding = Embedding(numwords, options.embedding_size, input_length=None)

embedded = embedding(input)

decoder_lstm = LSTM(options.lstm_capacity, return_sequences=True)
h = decoder_lstm(embedded)

if options.extra is not None:
    for _ in range(options.extra):
        h = LSTM(options.lstm_capacity, return_sequences=True)(h)

fromhidden = Dense(numwords, activation='linear')
out = TimeDistributed(fromhidden)(h)

model = Model(input, out)

opt = tf.keras.optimizers.Adam(learning_rate=options.lr)
lss = sparse_loss

model.compile(opt, lss)
model.summary()

## Training

epoch = 0
instances_seen = 0
while epoch < options.epochs:
    for batch in tqdm(x):
        n, l = batch.shape

        batch_shifted = np.concatenate([np.ones((n, 1)), batch], axis=1)  # prepend start symbol
        batch_out = np.concatenate([batch, np.zeros((n, 1))], axis=1)     # append pad symbol

        loss = model.train_on_batch(batch_shifted, batch_out[:, :, None])

        instances_seen += n
        tbw.add_scalar('lm/batch-loss', float(loss), instances_seen)
    print(loss)
    epoch += 1



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_1 (Embedding)              │ (None, None, 300)           │       3,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, None, 256)           │         570,368 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, None, 10000)         │       2,570,000 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,140,368 (23.42 MB)

 Trainable params: 6,140,368 (23.42 MB)

 Non-trainable params: 0 (0.00 B)

...

4.817323


100%|██████████| 233/233 [00:10<00:00, 22.69it/s]


4.772934


100%|██████████| 233/233 [00:10<00:00, 22.67it/s]

4.7297025


# LSTM Models Training

In [ ]:
def create_model(numwords, embedding_size, lstm_capacity, extra_layer=False, reverse=False):
    """
    Create a model with specified parameters

    Args:
        numwords: vocabulary size
        embedding_size: size of word embeddings
        lstm_capacity: number of LSTM units
        extra_layer: whether to add a second LSTM layer
        reverse: whether to process sequence in reverse
    """
    input = Input(shape=(None,))
    embedding = Embedding(numwords, embedding_size, input_length=None)
    embedded = embedding(input)

    # First LSTM layer with go_backwards parameter
    h = LSTM(lstm_capacity, return_sequences=True, go_backwards=reverse)(embedded)

    # Optional second LSTM layer, also with go_backwards
    if extra_layer:
        h = LSTM(lstm_capacity, return_sequences=True, go_backwards=reverse)(h)

    fromhidden = Dense(numwords, activation='linear')
    out = TimeDistributed(fromhidden)(h)

    model = Model(input, out)
    opt = tf.keras.optimizers.Adam(learning_rate=options.lr)
    model.compile(opt, sparse_loss)

    return model

# The rest of the code remains the same, but now we don't need any sequence reversal in the training function
def train_model(model, train_data, val_data, epochs, reverse=False):
    """
    Train the model and return training history
    """
    epoch = 0
    instances_seen = 0
    training_losses = []
    validation_losses = []

    while epoch < epochs:
        epoch_losses = []
        print(f'Epoch {epoch+1}/{epochs}')

        for batch in tqdm(train_data):
            n, l = batch.shape
            batch_shifted = np.concatenate([np.ones((n, 1)), batch], axis=1)
            batch_out = np.concatenate([batch, np.zeros((n, 1))], axis=1)

            loss = model.train_on_batch(batch_shifted, batch_out[:, :, None])
            epoch_losses.append(loss)

            instances_seen += n
            tbw.add_scalar('lm/batch-loss', float(loss), instances_seen)

        avg_epoch_loss = np.mean(epoch_losses)
        training_losses.append(avg_epoch_loss)
        print(f'Average training loss: {avg_epoch_loss}')

        val_losses = []
        for batch in val_data:
            n, l = batch.shape
            batch_shifted = np.concatenate([np.ones((n, 1)), batch], axis=1)
            batch_out = np.concatenate([batch, np.zeros((n, 1))], axis=1)

            val_loss = model.test_on_batch(batch_shifted, batch_out[:, :, None])
            val_losses.append(val_loss)

        avg_val_loss = np.mean(val_losses)
        validation_losses.append(avg_val_loss)
        print(f'Validation loss: {avg_val_loss}')

        epoch += 1

    return training_losses, validation_losses

# Create and train all 4 model configurations
models_config = [
    {'name': '1layer_forward', 'extra_layer': False, 'reverse': False},
    {'name': '1layer_reverse', 'extra_layer': False, 'reverse': True},
    {'name': '2layer_forward', 'extra_layer': True, 'reverse': False},
    {'name': '2layer_reverse', 'extra_layer': True, 'reverse': True}
]

# Dictionary to store all models and their training histories
results = {}

for config in models_config:
    print(f"\nTraining {config['name']}...")

    model = create_model(
        numwords=numwords,
        embedding_size=options.embedding_size,
        lstm_capacity=options.lstm_capacity,
        extra_layer=config['extra_layer'],
        reverse=config['reverse']
    )

    training_losses, validation_losses = train_model(
        model=model,
        train_data=train_data,
        val_data=val_data,
        epochs=options.epochs,
        reverse=config['reverse']
    )

    results[config['name']] = {
        'model': model,
        'training_losses': training_losses,
        'validation_losses': validation_losses
    }


Training 1layer_forward...
Epoch 1/20


100%|██████████| 186/186 [00:09<00:00, 19.54it/s]


Average training loss: 7.15544319152832
Validation loss: 6.525925159454346
Epoch 2/20


100%|██████████| 186/186 [00:07<00:00, 25.85it/s]


Average training loss: 6.330569267272949
Validation loss: 6.188929080963135
Epoch 3/20


...

Average training loss: 2.840996265411377
Validation loss: 2.7876527309417725
Epoch 19/20


100%|██████████| 186/186 [00:08<00:00, 22.09it/s]


Average training loss: 2.73661732673645
Validation loss: 2.6870157718658447
Epoch 20/20


100%|██████████| 186/186 [00:08<00:00, 22.18it/s]


Average training loss: 2.6393094062805176
Validation loss: 2.5928659439086914


#Tasks 2,5,6,8,9

In [ ]:
def calculate_sentence_probability(model, sentence, w2i):
    """
    Calculate the probability of a sentence according to the model.

    Args:
        model: trained Keras model
        sentence: string of words
        w2i: word to index dictionary

    Returns:
        probability of the sentence
    """
    # Convert sentence to indices
    words = sentence.lower().split()
    indices = [w2i.get(word, w2i['<UNK>']) for word in words]

    # Prepare input sequence
    input_seq = np.array([1] + indices[:-1])[None, :]  # Add start token and batch dimension
    target_seq = np.array(indices)[None, :, None]  # Add batch and feature dimensions

    # Get model predictions
    predictions = model.predict(input_seq, verbose=0)

    # Calculate probability as product of individual word probabilities
    log_prob = 0
    for i, word_idx in enumerate(indices):
        # Get softmax probabilities for current position
        logits = predictions[0, i]
        probs = np.exp(logits - np.max(logits))
        probs = probs / np.sum(probs)

        # Add log probability of current word
        log_prob += np.log(probs[word_idx] + 1e-10)

    return np.exp(log_prob)

def deconstruct_sentence(model, start_words, length, temperature, w2i, i2w):
    """
    Generate sentences backwards from given start words at different temperatures.

    Args:
        model: trained Keras model
        start_words: string of starting words
        length: desired total length of sentence
        temperature: sampling temperature
        w2i: word to index dictionary
        i2w: index to word dictionary

    Returns:
        generated sentence
    """
    words = start_words.lower().split()
    current_length = len(words)

    # Convert initial words to indices
    indices = [w2i.get(word, w2i['<UNK>']) for word in words]

    while current_length < length:
        # Prepare input sequence
        input_seq = np.array([1] + indices)[None, :]  # Add start token and batch dimension

        # Get model predictions
        predictions = model.predict(input_seq, verbose=0)
        next_word_logits = predictions[0, len(indices)]

        # Apply temperature
        if temperature == 0:
            next_word_idx = np.argmax(next_word_logits)
        else:
            scaled_logits = next_word_logits / temperature
            probs = np.exp(scaled_logits - np.max(scaled_logits))
            probs = probs / np.sum(probs)
            next_word_idx = np.random.choice(len(probs), p=probs)

        indices.append(next_word_idx)
        words.append(i2w[next_word_idx])
        current_length += 1

    return ' '.join(words)

# Calculate perplexity for all models on all datasets
def calculate_perplexity(model, data):
    """Calculate perplexity for a model on given data."""
    total_loss = 0
    total_samples = 0

    for batch in data:
        n, l = batch.shape
        batch_shifted = np.concatenate([np.ones((n, 1)), batch], axis=1)
        batch_out = np.concatenate([batch, np.zeros((n, 1))], axis=1)

        loss = model.test_on_batch(batch_shifted, batch_out[:, :, None])
        total_loss += loss * n
        total_samples += n

    avg_loss = total_loss / total_samples
    return np.exp(avg_loss)

# Now let's run all the analyses

# 1. Calculate perplexity for all models on all datasets
print("\nPerplexity Results:")
print("Model\t\tTrain\tValidation\tTest")
for config in models_config:
    name = config['name']
    model = results[name]['model']

    train_perp = calculate_perplexity(model, train_data)
    val_perp = calculate_perplexity(model, val_data)
    test_perp = calculate_perplexity(model, test_data)

    print(f"{name}\t{train_perp:.2f}\t{val_perp:.2f}\t{test_perp:.2f}")

# 2. Generate sentences at different temperatures
print("\nSentence Generation at Different Temperatures:")
start_words = "love I"
for temp in [0.1, 1.0, 10.0]:
    print(f"\nTemperature: {temp}")
    for config in models_config:
        name = config['name']
        model = results[name]['model']
        sentence = deconstruct_sentence(model, start_words, 7, temp, w2i, i2w)
        print(f"{name}: {sentence}")

# 3. Calculate probabilities for generated sentences and "love i cupcakes"
print("\nSentence Probabilities:")
sentences_to_check = []
# Add generated sentences
for temp in [0.1, 1.0, 10.0]:
    for config in models_config:
        name = config['name']
        model = results[name]['model']
        sentence = deconstruct_sentence(model, start_words, 7, temp, w2i, i2w)
        sentences_to_check.append(sentence)
# Add additional sentence
sentences_to_check.append("love i cupcakes")

for sentence in sentences_to_check:
    print(f"\nProbabilities for: {sentence}")
    for config in models_config:
        name = config['name']
        model = results[name]['model']
        prob = calculate_sentence_probability(model, sentence, w2i)
        print(f"{name}: {prob:.2e}")


Perplexity Results:
Model		Train	Validation	Test
1layer_forward	105.69	104.90	105.20
1layer_reverse	231.39	229.91	230.92
2layer_forward	255.51	253.78	253.95
2layer_reverse	7.15	7.01	6.99

Sentence Generation at Different Temperatures:

Temperature: 0.1
1layer_forward: love i of the <UNK> lrb <UNK>
1layer_reverse: love i <PAD> <PAD> <PAD> <PAD> <PAD>
2layer_forward: love i is a <UNK> of the
2layer_reverse: love i <PAD> <PAD> <PAD> <PAD> <PAD>

Temperature: 1.0
1layer_forward: love i <UNK> leaves '' <UNK> weapon
1layer_reverse: love i <PAD> <PAD> <PAD> <PAD> <PAD>
2layer_forward: love i family latd marrow or al
2layer_reverse: love i days pp oil philosopher khan

Temperature: 10.0
1layer_forward: love i 1992 physicians joel lawyer incorrectly
1layer_reverse: love i million it inverness remaining louis
2layer_forward: love i accounts zack zeta manipulation intelligence
2layer_reverse: love i country italy teaching basilica tenor

Sentence Probabilities:

Probabilities for: love i of the 

#7. Predict next word

In [ ]:
def get_next_word_prediction(model, w2i, i2w, word):
    """
    Given a trained model, vocabulary mappings (w2i, i2w), and an input word,
    predict the next word and return the prediction with probabilities.
    """
    word = word.lower().strip()

    if word not in w2i:
        return f"Word '{word}' not in vocabulary. Please try another word."

    word_idx = w2i[word]
    input_seq = np.array([[1, word_idx]])  # Add batch dimension

    try:
        # Use model.predict to get predictions
        predictions = model.predict(input_seq, verbose=0)
        next_word_logits = predictions[0, 1]

        # Apply temperature to logits
        exp_logits = np.exp(next_word_logits - np.max(next_word_logits))
        probs = exp_logits / np.sum(exp_logits)

        # Get top 5 predictions
        top_indices = np.argsort(probs)[-5:][::-1]

        result = "\nTop 5 predicted next words:\n"
        for idx in top_indices:
            result += f"{i2w[idx]}: {probs[idx]:.3f}\n"
        return result
    except Exception as e:
        return f"Error during prediction: {str(e)}"

# Example of how to call it manually:
# Replace these with your actual word-to-index and index-to-word dictionaries and model
word = "love"
model = results['2layer_reverse']['model']  # Use the model you want
output = get_next_word_prediction(model, w2i, i2w, word)
print(output)


Top 5 predicted next words:
core: 0.024
basketball: 0.017
priest: 0.012
distance: 0.012
events: 0.011

